# Lab 4
+ ## Автор: Роман Кривохижа
+ ## Група: ІС-72
+ ## Викладач: Новікова П.А.

****
****
****

## Module importing

In [2]:
import numpy as np
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
sns.set_style('darkgrid')
%matplotlib inline

## Algorithm implementation

+ Алгоритм, який відтворює функціонування системи, за допомогою комп’ютерної програми називається **алгоритмом імітації**.

+ Вимоги на обслуговування генеруються елементом **CREATE** і відправляються на обслуговування до елементу **PROCESS**, який здійснює обслуговування з часовою затримкою, заданою випадковим числом, та обмеження на довжину черги, заданим невід’ємним числом

+ Імітація здійснюється за відомим з попередньої теми принципом: визначається момент найближчої події, просувається час в момент найближчої події та здійснюється відповідна подія.

<font size="4">Ймовірність відмови: $P = \frac{N_{unserv}}{N_{all}}$</font>

<font size="4">Середня довжина черги: $L_{aver} = \frac{\sum_{i} L_i \Delta t_i}{T_{mod}}$</font>

<font size="4">Середній час очікування: $Q_{aver} = \frac{\sum_{i} L_i \Delta t_i}{N_{serv}}$</font>

## Задача 1

In [5]:
class Rand:
    """
    Генерація випадкового числа за заданим законом розподілу
    """
    @staticmethod
    def exp(time_mean):
        a = 0
        while a == 0:
            a = np.random.rand()
        return -time_mean * np.log(a)
    
    @staticmethod
    def unif(time_min, time_max):
        a = 0
        while a == 0:
            a = np.random.rand()
        a = time_min + a * (time_max - time_min)
        return a
    
    @staticmethod
    def norm(time_mean, time_deviation):
        return np.random.normal(loc=time_mean, scale=time_deviation)

In [6]:
class Element:
    id_curr = 0
    
    def __init__(self, name=None, delay_mean=1., delay_dev=0., distribution='', p=None, n_channel=1):
        self.n_channel = n_channel
        self.tnexts = [0.0]*self.n_channel
        self.delay_mean = delay_mean
        self.delay_dev = delay_dev
        self.distribution = distribution
        self.quantity = 0
        self.tcurr = 0
        self.states = [0]*self.n_channel
        self.next_elements = None
        self.p = p
        self.id_curr = Element.id_curr
        Element.id_curr += 1
        self.name = f'element_{self.id_curr}' if name is None else name
        
    def get_delay(self):
        if self.distribution == 'exp':
            return Rand.exp(self.delay_mean)
        elif self.distribution == 'unif':
            return Rand.unif(self.delay_mean, self.delay_dev)
        elif self.distribution == 'norm':
            return Rand.norm(self.delay_mean, self.delay_dev)
        else:
            return self.delay_mean
        
    def in_act(self):
        pass
    
    def out_act(self):
        self.quantity += 1
        
    def print_info(self):
        print(f'{self.name}: state={self.states}; quantity={self.quantity}; tnext={np.round(self.tnexts, 5)}')
        
    def print_result(self):
        print(f'{self.name}: state={self.states}; quantity={self.quantity};\n')
    
    def do_statistics(self, delta):
        pass
    
    def find_free_channels(self):
        res = []
        for i in range(self.n_channel):
            if self.states[i] == 0:
                res.append(i)
        return res
        
    def find_current_channels(self):
        res = []
        for i in range(self.n_channel):
            if self.tnexts[i] == self.tcurr:
                res.append(i)
        return res

In [7]:
class Create(Element):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def out_act(self):
        # виконуємо збільшення лічильника кількості
        super().out_act()
        # встановлюємо коли пристрій буде вільним
        self.tnexts[0] = self.tcurr + super().get_delay()
        # передаємо створену заявку на настпуні елемент
        p1 = self.next_elements[0]
        p2 = self.next_elements[1]
        if p1.queue == p2.queue:
            p1.in_act()
        elif p1.queue < p2.queue:
            p1.in_act()
        else:
            p2.in_act()

In [8]:
class Despose(Element):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.tnexts = [np.inf]
        
    def in_act(self):
        # виконуємо збільшення лічильника кількості
        super().out_act()
        
    def out_act(self):
        pass

In [33]:
class Process(Element):
    def __init__(self, maxqueue=np.inf, n_channel=1, **kwargs):
        super().__init__(**kwargs)
        self.queue = 0
        self.n_channel = n_channel
        self.maxqueue = maxqueue
        self.mean_queue_length = self.queue
        self.max_obs_queue_length = self.queue
        self.failure = 0
        self.tnexts = [np.inf]*n_channel
        self.states = [0]*n_channel
        self.delta_t_arrive = 0
        self.tprev_arrive = 0
        self.delta_t_in_bank = 0
        self.tprev_in_bank = 0
        
    def in_act(self):
        # перевіряємо чи вільний пристрій
        free_channels = self.find_free_channels()
        for i in free_channels:
            self.tprev_in_bank = self.tcurr
            # позначаємо що пристрій зайнятий
            self.states[i] = 1
            # встановлюємо коли пристрій буде вільним
            self.tnexts[i] = self.tcurr + super().get_delay()
            break
        else:
            if self.queue < self.maxqueue:
                self.queue += 1
                if self.queue > self.max_obs_queue_length:
                    self.max_obs_queue_length = self.queue
            else:
                self.failure += 1
            
    def out_act(self):
        # виконуємо збільшення лічильника кількості
        super().out_act()
        
        current_channels = self.find_current_channels()
        
        for i in current_channels:
            # позначаємо, що елемент вільний
            self.tnexts[i] = np.inf
            self.states[i] = 0
            
            # обраховуємо середній інтервал часу між від'їздами клієнтів від вікон
            self.delta_t_arrive =+ self.tcurr - self.tprev_arrive
            self.tprev_arrive = self.tcurr
            
            # обраховуємо середній час перебування клієнта в банку
            self.delta_t_in_bank =+ self.tcurr - self.tprev_in_bank
            # дістаємо з черги елемент, якщо він там є
            if self.queue > 0:
                self.queue -= 1
                self.states[i] = 1
                self.tnexts[i] = self.tcurr + super().get_delay()
            elif self.next_elements is not None:
                next_element = np.random.choice(self.next_elements, p=self.p)
                next_element.in_act()
        
    def print_info(self):
        super().print_info()
        print(f'queue={self.queue}; failure={self.failure}')
        
    def do_statistics(self, delta):
        self.mean_queue_length =+ delta * self.queue

In [34]:
class Model():
    def __init__(self, elements=[], print_logs=False, balancing=None):
        self.elements = elements
        self.tnext = 0
        self.tcurr = self.tnext
        self.event = elements[0]
        self.print_logs = print_logs
        self.balancing = balancing
        self.change_queue = 0
        self.average_cnt_client_in_bank = 0
        
    def balance_elements(self):
        p1 = self.balancing[0]
        p2 = self.balancing[1]
        
        if np.abs(p1.queue - p2.queue) >= 2:
            self.change_queue += 1
            if p1.queue < p2.queue:
                p1.queue += 1
                p2.queue -= 1
            else:
                p1.queue -= 1
                p2.queue += 1
                
    def calc_average_cnt_client_in_bank(self, delta):
        self.average_cnt_client_in_bank += delta * (self.balancing[0].queue + self.balancing[1].queue + self.balancing[0].states[0] + self.balancing[1].states[0])
        
    def simulate(self, time):
        self.max_time = time
        while self.tcurr < self.max_time:
            self.tnext = np.inf
            for e in self.elements:
                # знайти найменший з моментів часу
                t_next_min = np.min(e.tnexts)
                if t_next_min < self.tnext and not isinstance(e, Despose):
                    self.tnext = t_next_min
                    self.event = e
            if self.print_logs:
                print(f'\nIt`s time for event in {self.event.name}; time={np.round(self.tnext, 5)}')
            
            # обраховуємо статистики
            for e in self.elements:
                e.do_statistics(self.tnext - self.tcurr)
            self.calc_average_cnt_client_in_bank(self.tnext - self.tcurr)   
            
            # робимо переміщення до моменту завершення
            self.tcurr = self.tnext
            for e in self.elements:
                e.tcurr = self.tcurr
            
            # виконуємо операцію завершення
            self.event.out_act()
            
            # Щоб зменшити обсяг обчислень, введемо також здійснення відповідної події для всіх елементів,
            # час наступної події яких співпадає з поточним моментом часу
            for e in self.elements:
                if self.tcurr in e.tnexts:
                    e.out_act()
            self.balance_elements()
            if self.print_logs:        
                self.print_info()
        return self.print_result()
        
    def print_info(self):
        for e in self.elements:
            e.print_info()
            
    def print_result(self):
        print('-----RESULT-----')
        
        global_max_observed_queue_length = 0
        global_mean_queue_length_accumulator = 0 
        global_failure_probability_accumulator = 0
        global_max_load = 0
        global_mean_load_accumulator = 0
        global_mean_time_of_arrivings_accumulator = 0
        global_mean_time_in_bank_accumulator = 0
        num_of_processors = 0
        
        for e in self.elements:
            e.print_result()
            if isinstance(e, Process):
                num_of_processors += 1
                mean_queue_length = e.mean_queue_length / self.tcurr
                failure_probability = e.failure / (e.quantity + e.failure) if (e.quantity + e.failure) != 0 else 0
                mean_load = e.quantity / self.max_time
                
                global_mean_queue_length_accumulator += mean_queue_length
                global_failure_probability_accumulator += failure_probability
                global_mean_load_accumulator += mean_load
                global_mean_time_of_arrivings_accumulator += e.delta_t_arrive / e.quantity
                global_mean_time_in_bank_accumulator += e.delta_t_in_bank / e.quantity
                
                if e.max_obs_queue_length > global_max_observed_queue_length:
                    global_max_observed_queue_length = e.max_obs_queue_length
                    
                if mean_load > global_max_load:
                    global_max_load = mean_load
                
                if self.print_logs:
#                     print(f"Average queue length: {mean_queue_length}")
#                     print(f"Failure probability: {failure_probability}")
                    print(f"Cashier mean load: {mean_load}")
                    print()
                
        global_mean_queue_length = global_mean_queue_length_accumulator / num_of_processors
        global_failure_probability = global_failure_probability_accumulator / num_of_processors
        global_mean_load = global_mean_load_accumulator / num_of_processors
        average_cnt_client_in_bank = self.average_cnt_client_in_bank / self.tcurr
        global_mean_time_of_arrivings = global_mean_time_of_arrivings_accumulator / num_of_processors
        global_mean_time_in_bank = global_mean_time_in_bank_accumulator / num_of_processors
        
        
        if self.print_logs:
            print(f"1. Global mean load: {global_mean_load}")
            print(f"2. Global mean client cnt in bank: {average_cnt_client_in_bank}")
            print(f"3. Global mean time of arrivings: {global_mean_time_of_arrivings}")
            print(f"4. Global mean time in bank: {global_mean_time_in_bank}")
            print(f"5. Global mean queue length: {global_mean_queue_length}")
            print(f"6. Global failure probability: {global_failure_probability}")
            print(f"7. Global change queue cnt: {self.change_queue}")
            print()
        
        return {
            "global_max_observed_queue_length": global_max_observed_queue_length,
            "global_mean_queue_length": global_mean_queue_length,
            "global_failure_probability": global_failure_probability,
            "global_max_load": global_max_load,
            "global_mean_load": global_mean_load
        }

In [35]:
class Task1():
    def __init__(self):
        c1 = Create(delay_mean=0.5, name='CREATOR1', distribution='exp')
        p1 = Process(maxqueue=3, delay_mean=0.3, name='CASHIER1', distribution='exp')
        p2 = Process(maxqueue=3, delay_mean=0.3, name='CASHIER2', distribution='exp')
        
        c1.next_elements = [p1, p2]
        
        elements = [c1, p1, p2]

        model = Model(elements, print_logs=True, balancing=[p1, p2])
        model.simulate(10)

In [36]:
Element.id_curr = 0
t1 = Task1()


It`s time for event in CREATOR1; time=0.0
CREATOR1: state=[0]; quantity=1; tnext=[0.11371]
CASHIER1: state=[1]; quantity=0; tnext=[0.37177]
queue=0; failure=0
CASHIER2: state=[0]; quantity=0; tnext=[inf]
queue=0; failure=0

It`s time for event in CREATOR1; time=0.11371
CREATOR1: state=[0]; quantity=2; tnext=[0.3862]
CASHIER1: state=[1]; quantity=0; tnext=[0.37177]
queue=1; failure=0
CASHIER2: state=[0]; quantity=0; tnext=[inf]
queue=0; failure=0

It`s time for event in CASHIER1; time=0.37177
CREATOR1: state=[0]; quantity=2; tnext=[0.3862]
CASHIER1: state=[1]; quantity=1; tnext=[0.51823]
queue=0; failure=0
CASHIER2: state=[0]; quantity=0; tnext=[inf]
queue=0; failure=0

It`s time for event in CREATOR1; time=0.3862
CREATOR1: state=[0]; quantity=3; tnext=[0.54355]
CASHIER1: state=[1]; quantity=1; tnext=[0.51823]
queue=1; failure=0
CASHIER2: state=[0]; quantity=0; tnext=[inf]
queue=0; failure=0

It`s time for event in CASHIER1; time=0.51823
CREATOR1: state=[0]; quantity=3; tnext=[0.54355]

## Задача 2

In [39]:
class Element:
    id_curr = 0
    
    def __init__(self, name=None, delay_mean=1., delay_dev=0., distribution='', p=None, n_channel=1):
        self.n_channel = n_channel
        self.tnexts = [0.0]*self.n_channel
        self.delay_mean = delay_mean
        self.delay_dev = delay_dev
        self.distribution = distribution
        self.quantity = 0
        self.tcurr = 0
        self.states = [0]*self.n_channel
        self.next_elements = None
        self.p = p
        self.id_curr = Element.id_curr
        Element.id_curr += 1
        self.name = f'element_{self.id_curr}' if name is None else name
        
    def get_delay(self):
        if self.distribution == 'exp':
            return Rand.exp(self.delay_mean)
        elif self.distribution == 'unif':
            return Rand.unif(self.delay_mean, self.delay_dev)
        elif self.distribution == 'norm':
            return Rand.norm(self.delay_mean, self.delay_dev)
        else:
            return self.delay_mean
        
    def in_act(self):
        pass
    
    def out_act(self):
        self.quantity += 1
        
    def print_info(self):
        print(f'{self.name}: state={self.states}; quantity={self.quantity}; tnext={np.round(self.tnexts, 5)}')
        
    def print_result(self):
        print(f'{self.name}: state={self.states}; quantity={self.quantity};\n')
    
    def do_statistics(self, delta):
        pass
    
    def find_free_channels(self):
        res = []
        for i in range(self.n_channel):
            if self.states[i] == 0:
                res.append(i)
        return res
        
    def find_current_channels(self):
        res = []
        for i in range(self.n_channel):
            if self.tnexts[i] == self.tcurr:
                res.append(i)
        return res

In [40]:
class Create(Element):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def out_act(self):
        # виконуємо збільшення лічильника кількості
        super().out_act()
        # встановлюємо коли пристрій буде вільним
        self.tnexts[0] = self.tcurr + super().get_delay()
        # передаємо створену заявку на настпуні елемент
        next_element = np.random.choice(self.next_elements, p=self.p)
        next_element.in_act()

In [41]:
class Despose(Element):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.tnexts = [np.inf]
        
    def in_act(self):
        # виконуємо збільшення лічильника кількості
        super().out_act()
        
    def out_act(self):
        pass

In [42]:
class Process(Element):
    def __init__(self, maxqueue=np.inf, n_channel=1, **kwargs):
        super().__init__(**kwargs)
        self.queue = 0
        self.n_channel = n_channel
        self.maxqueue = maxqueue
        self.mean_queue_length = self.queue
        self.max_obs_queue_length = self.queue
        self.failure = 0
        self.tnexts = [np.inf]*n_channel
        self.states = [0]*n_channel
        
    def in_act(self):
        # перевіряємо чи вільний пристрій
        free_channels = self.find_free_channels()
        for i in free_channels:
            # позначаємо що пристрій зайнятий
            self.states[i] = 1
            # встановлюємо коли пристрій буде вільним
            self.tnexts[i] = self.tcurr + super().get_delay()
            break
        else:
            if self.queue < self.maxqueue:
                self.queue += 1
                if self.queue > self.max_obs_queue_length:
                    self.max_obs_queue_length = self.queue
            else:
                self.failure += 1
            
    def out_act(self):
        # виконуємо збільшення лічильника кількості
        super().out_act()
        
        current_channels = self.find_current_channels()
        
        for i in current_channels:
            # позначаємо, що елемент вільний
            self.tnexts[i] = np.inf
            self.states[i] = 0

            # дістаємо з черги елемент, якщо він там є
            if self.queue > 0:
                self.queue -= 1
                self.states[i] = 1
                self.tnexts[i] = self.tcurr + super().get_delay()
            elif self.next_elements is not None:
                next_element = np.random.choice(self.next_elements, p=self.p)
                next_element.in_act()
        
    def print_info(self):
        super().print_info()
        print(f'failure={self.failure}')
        
    def do_statistics(self, delta):
        self.mean_queue_length =+ delta * self.queue

In [43]:
class Model():
    def __init__(self, elements=[], print_logs=False):
        self.elements = elements
        self.tnext = 0
        self.tcurr = self.tnext
        self.event = elements[0]
        self.print_logs = print_logs
        
    def simulate(self, time):
        self.max_time = time
        while self.tcurr < self.max_time:
            self.tnext = np.inf
            for e in self.elements:
                # знайти найменший з моментів часу
                t_next_min = np.min(e.tnexts)
                if t_next_min < self.tnext and not isinstance(e, Despose):
                    self.tnext = t_next_min
                    self.event = e
            if self.print_logs:
                print(f'\nIt`s time for event in {self.event.name}; time={np.round(self.tnext, 5)}')
            
            # обраховуємо статистики
            for e in self.elements:
                e.do_statistics(self.tnext - self.tcurr)
                
            # робимо переміщення до моменту завершення
            self.tcurr = self.tnext
            for e in self.elements:
                e.tcurr = self.tcurr
            
            # виконуємо операцію завершення
            self.event.out_act()
            
            # Щоб зменшити обсяг обчислень, введемо також здійснення відповідної події для всіх елементів,
            # час наступної події яких співпадає з поточним моментом часу
            for e in self.elements:
                if self.tcurr in e.tnexts:
                    e.out_act()
            if self.print_logs:        
                self.print_info()
        return self.print_result()
        
    def print_info(self):
        for e in self.elements:
            e.print_info()
            
    def print_result(self):
        print('-----RESULT-----')
        
        global_max_observed_queue_length = 0
        global_mean_queue_length_accumulator = 0
        global_failure_probability_accumulator = 0
        global_max_load = 0
        global_mean_load_accumulator = 0
        num_of_processors = 0
        
        for e in self.elements:
            e.print_result()
            if isinstance(e, Process):
                num_of_processors += 1
                mean_queue_length = e.mean_queue_length / self.tcurr
                failure_probability = e.failure / (e.quantity + e.failure) if (e.quantity + e.failure) != 0 else 0
                mean_load = e.quantity / self.max_time
                
                global_mean_queue_length_accumulator += mean_queue_length
                global_failure_probability_accumulator += failure_probability
                global_mean_load_accumulator += mean_load
                
                if e.max_obs_queue_length > global_max_observed_queue_length:
                    global_max_observed_queue_length = e.max_obs_queue_length
                    
                if mean_load > global_max_load:
                    global_max_load = mean_load
                
                if self.print_logs:
                    print(f"Average queue length: {mean_queue_length}")
                    print(f"Failure probability: {failure_probability}")
                    print(f"Average load: {mean_load}")
                    print()
                
        global_mean_queue_length = global_mean_queue_length_accumulator / num_of_processors
        global_failure_probability = global_failure_probability_accumulator / num_of_processors
        global_mean_load = global_mean_load_accumulator / num_of_processors
        
        if self.print_logs:
            print(f"Global max observed queue length: {global_max_observed_queue_length}")
            print(f"Global mean queue length: {global_mean_queue_length}")
            print(f"Global failure probability: {global_failure_probability}")
            print(f"Global max load: {global_max_load}")
            print(f"Global mean load: {global_mean_load}")
            print()
        
        return {
            "global_max_observed_queue_length": global_max_observed_queue_length,
            "global_mean_queue_length": global_mean_queue_length,
            "global_failure_probability": global_failure_probability,
            "global_max_load": global_max_load,
            "global_mean_load": global_mean_load
        }

In [44]:
class Task2():
    def __init__(self):
        c1 = Create(delay_mean=1.0, name='CREATOR_1', distribution='exp')
        p1 = Process(maxqueue=100, delay_mean=1.0, n_channel=2, name='RECEPTION', distribution='exp')
        p2 = Process(maxqueue=100, delay_mean=1.0, n_channel=3, name='FOLLOWING_TO_THE_WARD', distribution='exp')
        p3 = Process(maxqueue=0, delay_mean=1.0, n_channel=100, name='FOLLOWING_TO_THE_LAB_RECEPTION1', distribution='exp')
        p4 = Process(maxqueue=100, delay_mean=1.0, n_channel=1, name='LAB_REGESTRY', distribution='exp')
        p5 = Process(maxqueue=100, delay_mean=1.0, n_channel=1, name='EXAMINATION', distribution='exp')
        p6 = Process(maxqueue=0, delay_mean=1.0, n_channel=100, name='FOLLOWING_TO_THE_LAB_RECEPTION2', distribution='exp')

        c1.next_elements = [p1]
        p1.next_elements = [p2, p3]
        p3.next_elements = [p4]
        p4.next_elements = [p5]
        p5.next_elements = [p6]
        p6.next_elements = [p1]
        
        elements = [c1, p1, p2, p3, p4, p5, p6]
        
        model = Model(elements, print_logs=True)
        model.simulate(10)

In [46]:
Element.id_curr = 0
t2 = Task2()


It`s time for event in CREATOR_1; time=0.0
CREATOR_1: state=[0]; quantity=1; tnext=[1.32103]
RECEPTION: state=[1, 0]; quantity=0; tnext=[0.0283    inf]
failure=0
FOLLOWING_TO_THE_WARD: state=[0, 0, 0]; quantity=0; tnext=[inf inf inf]
failure=0
FOLLOWING_TO_THE_LAB_RECEPTION1: state=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]; quantity=0; tnext=[inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf in

## Conclusion

**В данній лабораторній роботі ми реалізували алгоритм імітації простої моделі обслуговування, використовуючи спосіб, який орієнтований на події (аби уникнути випадку, коли дві події приходять в один час).**
+ за допомогою статичних методів була зібрана інформація про роботу/поведінку моделі
+ зміна віхдних параметрів моделі призводить до зміни вихідних значень
+ при досить великій кількості вхідинх параметрів важко проводити етап верифікації моделі

P.S. нижче наведено декілька графіків, які ілюструють залежність вихідних параметрів від вхідних